## Настройка локального окружения

#### Загрузка переменных окружения

In [7]:
# Загрузка конфигурации и переменных окружения
from utils import Config, get_prompt_template, pretty_print_pydantic, save_general_state_to_markdown

from dotenv import load_dotenv

load_dotenv()

# Инициализация конфигурации
config = Config()
config.ensure_directories()

print("Конфигурация загружена успешно!")
print(f"Директория для данных: {config.main_dir}")
print(f"Модель: {config.get_model_name()}")

Конфигурация загружена успешно!
Директория для данных: ./data
Модель: gpt-4.1-mini


### Установка зависимостей

In [8]:
# Зависимости устанавливаются через poetry install
# Убедитесь, что выполнили: poetry install

### Импорты

In [9]:
import os
import time
import re
import requests
import json
import yaml
import datetime
import operator
import urllib.parse
from urllib.parse import urlparse
from ftfy import fix_text
import uuid
import builtins
from io import BytesIO

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore
from langchain_core.runnables.config import RunnableConfig
from langgraph.constants import Send
from langchain_core.utils.function_calling import convert_to_openai_tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import interrupt, Command

from IPython.display import display
from IPython.display import Image as IPythonImage

from langchain_community.document_loaders import WikipediaLoader
from langchain_community.tools import TavilySearchResults

from jinja2 import Template
from pydantic import BaseModel, Field
from typing import Annotated, Any, Optional, List, Literal, Union
from typing_extensions import TypedDict

import openai
from openai import OpenAI, Client

### Переменные окружения

In [10]:
from langfuse import get_client

langfuse = get_client()
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


In [11]:
from langfuse.langchain import CallbackHandler

langfuse_handler = CallbackHandler()

In [12]:
# Переменные окружения загружаются из .env файла через python-dotenv
# Убедитесь, что создали .env файл на основе env.example

# Проверяем наличие необходимых переменных окружения
required_env_vars = [
    'OPENAI_API_KEY',
    'TAVILY_API_KEY', 
]

missing_vars = []
for var in required_env_vars:
    if not os.getenv(var):
        missing_vars.append(var)

if missing_vars:
    print(f"⚠️ Отсутствуют переменные окружения: {', '.join(missing_vars)}")
    print("Создайте .env файл на основе env.example")
else:
    print("✅ Все необходимые переменные окружения загружены")

FIREWORKS_API_KEY = os.getenv("NEWS_FIREWORKS_API_KEY")

✅ Все необходимые переменные окружения загружены


In [13]:
# Конфигурация уже загружена через utils.Config
print(f"Основная директория: {config.main_dir}")
print(f"Путь к промптам: {config.prompts_config_path}")
print(f"Путь к конфигурации графа: {config.graph_config_path}")

Основная директория: ./data
Путь к промптам: ./config/prompts.yaml
Путь к конфигурации графа: ./config/graph.yaml


### Конфиги

### Константы

In [14]:
# Используем конфигурацию из utils
main_dir = config.main_dir
model_name = config.get_model_name()

print(f"Рабочая директория: {main_dir}")
print(f"Модель: {model_name}")

Рабочая директория: ./data
Модель: gpt-4.1-mini


## Решение

## Prompts

In [15]:
# Шаблоны промптов загружаются из конфигурации
GENERATING_CONTENT_SYSTEM_PROMPT_TEMPLATE = get_prompt_template('generating_content_system_prompt', config)
GEN_QUESTION_SYSTEM_PROMPT_TEMPLATE = get_prompt_template('gen_question_system_prompt', config)  
GEN_QUESTION_REFINE_SYSTEM_PROMPT_TEMPLATE = get_prompt_template('gen_question_refine_system_prompt', config)
GEN_ANSWER_SYSTEM_PROMPT_TEMPLATE = get_prompt_template('gen_answer_system_prompt', config)

print("✅ Промпты загружены из конфигурации")


✅ Промпты загружены из конфигурации


## LangGraph

In [16]:
def display_graph(app, xray=0):
  display(IPythonImage(app.get_graph(xray=xray).draw_mermaid_png()))

def pretty_print_pydantic(pydantic_model):
    return json.dumps(pydantic_model.model_json_schema(), indent=4)

In [17]:
from utils import GapQuestions, GeneralState, GapQuestionsHITL

In [18]:
model = ChatOpenAI(
    model=model_name,
    openai_api_key=os.getenv('OPENAI_API_KEY'),
)

In [19]:
async def generating_content_node(state: GeneralState) -> Command[Literal["generating_questions"]]:
    messages = [
    SystemMessage(
        content=GENERATING_CONTENT_SYSTEM_PROMPT_TEMPLATE.render(
            exam_question=state.exam_question,
            )
        )
    ]

    my_response = await model.ainvoke(messages)

    return Command(
        goto="generating_questions",
        update={"generated_material": my_response.content}
    )

async def generating_questions_node(state: GeneralState) -> Command[Literal["answer_question"]]:
    # Если первый запуск - генерируем начальные вопросы
    if not state.feedback_messages:
        messages = [
            SystemMessage(
                content=GEN_QUESTION_SYSTEM_PROMPT_TEMPLATE.render(
                    exam_question=state.exam_question,
                    study_material=state.generated_material,
                    json_schema=pretty_print_pydantic(GapQuestions)
                )
            )
        ]

        my_response = await model.with_structured_output(GapQuestions).ainvoke(messages)
        
        # Добавляем первое сообщение в историю обратной связи
        initial_message = AIMessage(content="\n".join([f"{i+1}. {q}" for i, q in enumerate(my_response.gap_questions)]))
        
        return Command(
            goto="generating_questions",
            update={
                "gap_questions": my_response.gap_questions,
                "feedback_messages": [initial_message]
            }
        )
    
    # Если контент уже есть - запрашиваем обратную связь
    messages_for_user = [state.feedback_messages[-1].content]
    
    # При первом запросе обратной связи добавляем подсказку
    if len(state.feedback_messages) == 1:
        messages_for_user.append("Оцените предложенные вопросы. Вы можете запросить изменения или подтвердить, что вопросы готовы к использованию.")
    
    # Прерываем выполнение и ждем ввода пользователя
    user_feedback = interrupt("\n".join(messages_for_user))
    
    # Генерируем улучшенный ответ с учетом обратной связи
    messages = [
        SystemMessage(
            content=GEN_QUESTION_REFINE_SYSTEM_PROMPT_TEMPLATE.render(
                exam_question=state.exam_question,
                study_material=state.generated_material,
                current_questions=state.gap_questions,
                json_schema=pretty_print_pydantic(GapQuestionsHITL)
            )
        )
    ]
    
    # Добавляем историю сообщений
    messages.extend(state.feedback_messages)
    messages.append(HumanMessage(content=user_feedback))
    
    # Получаем обновленные вопросы
    refined_response = await model.with_structured_output(GapQuestionsHITL).ainvoke(messages)
    
    # Создаем новое сообщение AI с обновленными вопросами
    refined_message = AIMessage("\n".join([f"{i+1}. {q}" for i, q in enumerate(refined_response.gap_questions)]))
    
    # Если модель решила, что вопросы готовы к финализации
    if refined_response.next_step == "finalize":
        parallel_sends = [
            Send("answer_question", {"question": question}) for question in refined_response.gap_questions
        ]
        
        return Command(
            goto=parallel_sends,
            update={
                "gap_questions": refined_response.gap_questions,
                "feedback_messages": [],
            }
        )
    else:
        # Продолжаем цикл уточнения
        return Command(
            goto="generating_questions",
            update={
                "gap_questions": refined_response.gap_questions,
                "feedback_messages": state.feedback_messages + [refined_message],
            }
        )

async def answer_question_node(data: str) -> Command[Literal["__end__"]]:
    messages = [
        SystemMessage(
        content=GEN_ANSWER_SYSTEM_PROMPT_TEMPLATE.render(
            exam_question=data['question'],
            )
        )
    ]

    my_response = await model.ainvoke(messages)

    return Command(
        goto=END,
        update={"gap_q_n_a": [f"##{data['question']}\n\n{my_response.content}"]}
    )

In [20]:
# --- Создание графа LangGraph ---
workflow = StateGraph(GeneralState)

# Добавляем узлы в граф
workflow.add_node("generating_content", generating_content_node)
workflow.add_node("generating_questions", generating_questions_node)
workflow.add_node("answer_question", answer_question_node)

workflow.add_edge(START, "generating_content")

In [26]:
checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)

print(app.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	generating_content(generating_content)
	generating_questions(generating_questions)
	answer_question(answer_question)
	__end__([<p>__end__</p>]):::last
	__start__ --> generating_content;
	answer_question -.-> __end__;
	generating_content -.-> generating_questions;
	generating_questions -.-> answer_question;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [27]:
thread_id = "1"
thread_config = {"configurable": {"thread_id": thread_id}}


questions = [
    "Классификация криптографических алгоритмов. Основные определения."
]

for i, question in enumerate(questions):
    print(f"\n{'='*50}")
    print(f"Обрабатываем вопрос {i+1}/{len(questions)}")
    print(f"{'='*50}")
    print(question)

    while True:
        state = app.get_state(thread_config)
        print(state)

        if state.values:
            input_state = Command(resume=user_feedback)
        else:
            input_state = {"exam_question": question}
        print(input_state)

        async for event in app.astream(
            input_state,
            thread_config,
            stream_mode="updates"
        ):
            for node_name, node_data in event.items():
                if type(node_data) == dict:
                    for key, value in node_data.items():
                        print(f"{key}: {value}")

        if '__interrupt__' in event:
            interrupt_message = event['__interrupt__'][0].value
            print(interrupt_message)
            
            # Создаем виджеты для ввода
            import ipywidgets as widgets
            from IPython.display import display
            
            text_input = widgets.Text(
                value='',
                placeholder='Введите ваш ответ',
                description='Ответ:',
                disabled=False,
                layout=widgets.Layout(width='80%')
            )
            
            button = widgets.Button(description="Продолжить")
            output = widgets.Output()
            
            # Создаем переменную для хранения ответа
            user_feedback_value = [None]
            
            def on_button_clicked(b):
                user_feedback_value[0] = text_input.value
                button.disabled = True
                text_input.disabled = True
                with output:
                    print(f"Получен ответ: {text_input.value}")
            
            button.on_click(on_button_clicked)
            
            # Отображаем виджеты
            display(text_input, button, output)
            
            # Ждем, пока пользователь не нажмет кнопку
            import time
            while user_feedback_value[0] is None:
                time.sleep(0.1)
            
            # Получаем введенный ответ
            user_feedback = user_feedback_value[0]
            print(f"Получен ответ: {user_feedback}")
        else:
            state = app.get_state(thread_id)
            md_text = save_general_state_to_markdown(state)
            
            # Сохраняем в директорию outputs
            output_path = os.path.join(main_dir, 'outputs', f'question_{i+1:02d}_{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.md')
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(md_text)
                
            print(f"✅ Результат сохранен: {output_path}")
            break
        

print(f"\n🎉 Обработка завершена! Результаты сохранены в {os.path.join(main_dir, 'outputs')}")


Обрабатываем вопрос 1/1
Классификация криптографических алгоритмов. Основные определения.
StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())
{'exam_question': 'Классификация криптографических алгоритмов. Основные определения.'}
generated_material: # Классификация криптографических алгоритмов. Основные определения

---

## Введение

В криптографии существует множество алгоритмов, предназначенных для обеспечения различных аспектов информационной безопасности: конфиденциальности, целостности, подлинности, отказоустойчивости и так далее. Каждая из этих задач решается своими методами и подходами, отличающимися по математической природе, структуре, свойствам и области применения.

Этот обзор призван систематизировать **классификацию криптографических алгоритмов** с точки зрения как теоретических основ, так и их прикладного использования, акцентируя внимание на глубине понимания, математи

Text(value='', description='Ответ:', layout=Layout(width='80%'), placeholder='Введите ваш ответ')

Button(description='Продолжить', style=ButtonStyle())

Output()

: 

: 